In [1]:
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader
from torchvision.transforms import transforms

import sys 
sys.path.append('.')


from cgiar.utils import get_dir
from cgiar.model import XCITMultipleMLP
from cgiar.data import CGIARDataset_V4, augmentations

# reduce font size of plots
plt.rcParams.update({'font.size': 8})

In [2]:
SEED=42
LR=1e-4
EPOCHS=30
IMAGE_SIZE=224
INITIAL_SIZE=512
TRAIN_BATCH_SIZE=64
TEST_BATCH_SIZE=32
HIDDEN_SIZE=512
NUM_FOLDS=5
NUM_VIEWS=10

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
DATA_DIR = get_dir('data/')
ARTIFACTS = get_dir('solutions/v10/#1/')

In [26]:
transform = transforms.Compose([
    transforms.RandomResizedCrop(IMAGE_SIZE),
    augmentations["RandomEqualize"],
    augmentations["RandomAffine"],
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [27]:
fold_idx=0

In [28]:
model = XCITMultipleMLP(
    model_name="xcit_nano_12_p16_224",
    pretrained=True,
    num_mlps=4,
    hidden_size=HIDDEN_SIZE
)

In [29]:
model.load_state_dict(torch.load(ARTIFACTS / f"model_fold_{fold_idx}.pth"))

<All keys matched successfully>

In [30]:
model = model.to(device)
model.eval()

XCITMultipleMLP(
  (model): Xcit(
    (patch_embed): ConvPatchEmbed(
      (proj): Sequential(
        (0): Sequential(
          (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): GELU(approximate='none')
        (2): Sequential(
          (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (3): GELU(approximate='none')
        (4): Sequential(
          (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (5): GELU(approximate='none')
        (6): Sequential(
          (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    

In [39]:
# Load test data frame from csv
X_test = pd.read_csv(DATA_DIR / 'Test.csv')

test_images = CGIARDataset_V4.load_images(X_test, DATA_DIR / "test", INITIAL_SIZE)
test_images = dict([test_images[idx] for idx in range(len(test_images))])

# Construct test dataset and dataloader
test_dataset = CGIARDataset_V4(
    images=test_images,
    num_views=NUM_VIEWS,
    transform=transform,
    features=X_test,
)
test_loader = DataLoader(
    test_dataset, 
    batch_size=TEST_BATCH_SIZE, 
    shuffle=False
)

In [40]:
len(test_dataset), len(test_loader)

(8663, 271)

In [45]:
from tqdm import tqdm

test_predictions = []

with torch.no_grad():
    
    for ids, images_list, growth_stage, season, _ in tqdm(test_loader):
    
        # average predictions from all the views
        outputs = torch.stack([model((
            growth_stage.to(device).squeeze(),
            season.to(device).squeeze(),
            images.to(device)
        )) for images in images_list]).mean(dim=0)
        
        # get predictions from all the folds
        outputs = outputs.tolist()
        test_predictions.extend(list(zip(ids, outputs)))

100%|██████████| 271/271 [04:56<00:00,  1.09s/it]


In [46]:
# load the sample submission file and update the extent column with the predictions
submission_df = pd.read_csv('data/SampleSubmission.csv')

# update the extent column with the predictions
submission_df['extent'] = submission_df['ID'].map(dict(test_predictions))

# save the submission file and trained model
submission_df.to_csv('submission.csv', index=False)